# Storing data in a BIG txt file

Data sources:
- Mozilla Common Voice

In [ ]:
%%capture
! pip install datasets

In [ ]:
! huggingface-cli login

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mozilla-foundation/common_voice_7_0", "it", use_auth_token=True, split="train")

with open("text.txt", "w") as f:
    f.write(" ".join(dataset["sentence"]))

# Training and storing KenLM-it-5

In [ ]:
! sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
! wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
! mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
! ls kenlm/build/bin

In [ ]:
! kenlm/build/bin/lmplz -o 5 < "text.txt" > "5gram-it-cv.arpa"

In [ ]:
with open("5gram-it-cv.arpa", "r") as read_file, open("5gram-it-cv-eos.arpa", "w") as write_file:
    has_added_eos = False
    for line in read_file:
        if not has_added_eos and "ngram 1=" in line:
            count=line.strip().split("=")[-1]
            write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
        elif not has_added_eos and "<s>" in line:
            write_file.write(line)
            write_file.write(line.replace("<s>", "</s>"))
            has_added_eos = True
        else:
            write_file.write(line)